## Prompt Engineering - LLM Generates Responses with Context from Testing Transcripts

In [ ]:
!pip install openai
!pip install transformers
import os
from openai import OpenAI
from transformers import GPT2Tokenizer
import json

In [ ]:
client = OpenAI(
    api_key= API_key,
)

def get_chat_completion(messages):
    response = client.chat.completions.create(
        model= LLM_Model,
        messages=messages,
        temperature=0.2,
        # max_tokens=800,
        top_p=0.5,
        frequency_penalty=0,
        # presence_penalty=0,
        stop=None
    )
    return response.choices[0].message.content

In [ ]:
# This is our prompt tip for the LLM to follow and learn how to be a professional leadership coach.
# We use iterative prompting to test each new version of our prompt tip and see for potential improvements of the model.
promp_emb_text = """
As a leadership coach, your primary role is to guide...
"""

In [ ]:
test_chat_message = [
    {"role": "system", "content": promp_emb_text}
]

In [ ]:
# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

max_tokens = 4096
responses_list = []

# The testing transcripts are the real transcripts of leadership coaching sessions happend between a human coach and a leader.
# We used them as context to feed into the LLM and let it generate the next response to the leader based on all the previous interactions.
with open('Testing Transcripts.json', 'r', encoding='utf-8') as file:
    conversations = json.load(file)

# Process each entry in the conversations
for entry in conversations:
        # Append both client and coach entries to the conversation history in test_chat_message
    if 'client' in entry:
        client_query = {'role': "user", "content": entry['client'] + "\nassistant:\n"}
        test_chat_message.append(client_query)
    if 'coach' in entry:
        test_chat_message.append({'role': "assistant", "content": entry['coach']})

    if 'client' in entry:
        response = get_chat_completion(test_chat_message)
        temp_res = {
            'prompt': client_query['content'],  # Use the formatted client query
            'response': response
        }
        responses_list.append(temp_res)

# Write LLM responses to a JSON file
with open('prompt1_test_response.json', 'w', encoding='utf-8') as file:
    json.dump(responses_list, file, ensure_ascii=False, indent=4)
